In [4]:
pip install -r requirements.txt


   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 2.1 MB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.2 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.2 MB 2.0 MB/s eta 0:00:02
   ------------ --------------------------- 1.3/4.2 MB 2.1 MB/s eta 0:00:02
   -------------- ------------------------- 1.6/4.2 MB 2.1 MB/s eta 0:00:02
   ------------------- -------------------- 2.1/4.2 MB 2.0 MB/s eta 0:00:02
   ---------------------- ----------------- 2.4/4.2 MB 2.0 MB/s eta 0:00:01
   --------------------------- ------------ 2.9/4.2 MB 2.1 MB/s eta 0:00:01
   ----------------------------- -------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:

!python src/agents/data_fetchers/newsapi_fetcher.py

Traceback (most recent call last):
  File "c:\Users\LukasDech\OneDrive - mesoneer\Documents\newsletter_department\src\agents\data_fetchers\newsapi_fetcher.py", line 11, in <module>
    from models.data_models import RawArticle # Unser Pydantic-Modell für Rohartikel
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'models'


In [ ]:
# main.py
# Haupt-Einstiegspunkt zum Starten der Newsletter-Pipeline.

from src.orchestrator import NewsletterOrchestrator
from src.utils.logging_setup import setup_logging # Sicherstellen, dass Logging früh konfiguriert wird
import logging # Importiere logging hier, um es direkt zu verwenden, falls nötig

if __name__ == "__main__":
    # Grundlegendes Logging für den Fall, dass der Orchestrator nicht initialisiert wird
    # oder Fehler vor dessen Logging-Setup auftreten.
    # Das Logging wird im Orchestrator detaillierter konfiguriert.
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__) # Logger für main.py

    logger.info("Starte den Newsletter-Generierungsprozess...")
    try:
        orchestrator = NewsletterOrchestrator()
        result_link = orchestrator.run_pipeline()
        
        if result_link:
            logger.info(f"Newsletter-Pipeline erfolgreich abgeschlossen. Newsletter verfügbar unter: {result_link}")
        else:
            logger.warning("Newsletter-Pipeline abgeschlossen, aber es wurde kein Link zum Newsletter generiert (möglicherweise Fehler oder keine Daten).")
    except Exception as e:
        logger.critical(f"Ein kritischer, nicht abgefangener Fehler ist in main.py aufgetreten: {e}", exc_info=True)
        # exc_info=True fügt den Traceback zum Log hinzu.

# src/orchestrator.py
# Steuert den gesamten Ablauf der Newsletter-Generierung.

import logging
from datetime import datetime, timezone # timezone hinzugefügt
from typing import List, Optional, Any, Dict # Dict hinzugefügt
import os

from src.utils.config_loader import load_env, get_env_variable # get_env_variable hinzugefügt
from src.utils.logging_setup import setup_logging

# Datenbeschaffer
from src.agents.data_fetchers.newsapi_fetcher import NewsAPIFetcher
# from src.agents.data_fetchers.rss_fetcher import RSSFetcher # Beispiel für weitere Fetcher
# from src.agents.data_fetchers.google_calendar_fetcher import GoogleCalendarEventFetcher, GoogleCalendarBirthdayFetcher # Beispiel
# from src.agents.data_fetchers.weather_fetcher import OpenWeatherMapFetcher # Beispiel

# LLM-Prozessoren
from src.agents.llm_processors.summarizer_agent import SummarizerAgent
from src.agents.llm_processors.categorizer_agent import CategorizerAgent
from src.agents.llm_processors.evaluator_agent import AudienceAlignmentAgent # NEUER AGENT

# Datenmodelle
from src.models.data_models import (
    RawArticle, ProcessedArticle, Event, Birthday, WeatherInfo, 
    NewsletterData, NewsletterSection
)

# Newsletter-Erstellung und Verteilung
from src.agents.newsletter_composer import NewsletterComposer
from src.agents.distributors.google_drive_uploader import GoogleDriveUploader


# LangSmith Konfiguration (wird automatisch von LangChain genutzt, wenn Umgebungsvariablen gesetzt sind)
logger = logging.getLogger(__name__)

class NewsletterOrchestrator:
    def __init__(self):
        load_env()
        # Hole LOG_LEVEL und LOG_FILE aus .env, mit Defaults
        log_level = get_env_variable("LOG_LEVEL", "INFO")
        log_file_path = get_env_variable("LOG_FILE") # Kann None sein
        setup_logging(log_level_str=log_level, log_file=log_file_path)
        logger.info("Initialisiere Newsletter Orchestrator...")

        self.news_fetchers = [
            NewsAPIFetcher(query="international OR world OR global", endpoint="everything", days_ago=1, source_name_override="Weltnachrichten (NewsAPI)"),
            NewsAPIFetcher(query="Technologie OR KI OR Innovation OR Wissenschaft", endpoint="everything", days_ago=1, source_name_override="Technologie & Wissenschaft (NewsAPI)"),
            NewsAPIFetcher(query="Wirtschaft OR Finanzen OR Börse", endpoint="everything", days_ago=1, source_name_override="Wirtschaft (NewsAPI)"),
            NewsAPIFetcher(query="Zürich OR Schweiz", endpoint="top-headlines", country="ch", category="general", source_name_override="Zürich/Schweiz Schlagzeilen (NewsAPI)"),
        ]
        
        # LLM-Prozessoren
        self.summarizer = SummarizerAgent()
        self.categorizer = CategorizerAgent(
            categories=[ 
                "IT & AI", "Welt und Politik", "Wirtschaft", 
                "Zürich Inside", "Kultur und Inspiration", "Der Rund um Blick"
            ]
        )
        # NEUER AGENT: AudienceAlignmentAgent
        # Das user_profile sollte idealerweise aus einer Konfigurationsdatei oder einer separaten Datei geladen werden.
        user_profile_text = get_env_variable("NEWSLETTER_USER_PROFILE", """
Dies ist ein Default-Profil: Leser interessiert sich für allgemeine Technologie-Neuigkeiten und wichtige globale Ereignisse.
Bevorzugt kurze, prägnante Zusammenfassungen. Weniger interessiert an Klatsch oder sehr nischigen Themen.
""") # Lade Profil aus .env oder nutze Default
        self.audience_aligner = AudienceAlignmentAgent(user_profile=user_profile_text)


    def _fetch_all_raw_articles(self) -> List[RawArticle]:
        all_raw_articles: List[RawArticle] = []
        for fetcher in self.news_fetchers:
            try:
                articles = fetcher.fetch_data()
                all_raw_articles.extend(articles)
                logger.info(f"{len(articles)} Artikel von '{fetcher.source_name}' abgerufen.")
            except Exception as e:
                logger.error(f"Fehler beim Abrufen von Artikeln von '{fetcher.source_name}': {e}", exc_info=True)
        return all_raw_articles

    def _process_articles_initial(self, raw_articles: List[RawArticle]) -> List[ProcessedArticle]:
        """Erste Verarbeitungsstufe: Zusammenfassen und Kategorisieren."""
        initial_processed_articles: List[ProcessedArticle] = []
        logger.info(f"Starte initiale Verarbeitung von {len(raw_articles)} Roh-Artikeln...")

        articles_to_process = raw_articles[:20] # Begrenze für Testzwecke
        logger.info(f"Verarbeite die ersten {len(articles_to_process)} Artikel mit Summarizer und Categorizer...")

        for i, raw_article in enumerate(articles_to_process):
            try:
                logger.debug(f"Initial verarbeite Artikel {i+1}/{len(articles_to_process)}: {raw_article.title}")
                summary = self.summarizer.summarize_article(raw_article)
                category = self.categorizer.categorize_article(raw_article, summary)
                
                initial_processed_articles.append(
                    ProcessedArticle(
                        title=raw_article.title or "Unbekannter Titel",
                        url=raw_article.url,
                        summary=summary,
                        category=category,
                        source_name=raw_article.source_name,
                        published_at=raw_article.published_at,
                        llm_processing_details={"initial_processing": "success"}
                    )
                )
            except Exception as e:
                logger.error(f"Fehler bei der initialen LLM-Verarbeitung für Artikel '{raw_article.title}': {e}", exc_info=True)
        
        logger.info(f"{len(initial_processed_articles)} Artikel initial mit LLMs verarbeitet.")
        return initial_processed_articles

    def _evaluate_and_filter_articles(self, articles_to_evaluate: List[ProcessedArticle]) -> List[ProcessedArticle]:
        """Zweite Verarbeitungsstufe: Evaluierung und Filterung."""
        final_approved_articles: List[ProcessedArticle] = []
        logger.info(f"Starte Evaluierung von {len(articles_to_evaluate)} vorverarbeiteten Artikeln...")

        # Schwellenwert für Relevanz-Score, könnte aus Konfiguration kommen
        relevance_threshold = float(get_env_variable("RELEVANCE_THRESHOLD", "0.5")) 

        for i, article in enumerate(articles_to_evaluate):
            try:
                logger.debug(f"Evaluiere Artikel {i+1}/{len(articles_to_evaluate)}: {article.title}")
                
                evaluation_result = self.audience_aligner.evaluate_article(article)
                
                # Stelle sicher, dass llm_processing_details existiert
                if article.llm_processing_details is None:
                    article.llm_processing_details = {}
                article.llm_processing_details["audience_evaluation"] = evaluation_result
                
                # Aktualisiere den relevance_score im Artikelobjekt
                article.relevance_score = evaluation_result.get("relevance_score", 0.0)

                logger.info(f"Artikel '{article.title}' Audience-Score: {article.relevance_score:.2f}. Feedback: {evaluation_result.get('feedback_summary', 'Kein Feedback')}")

                if article.relevance_score >= relevance_threshold:
                    final_approved_articles.append(article)
                else:
                    logger.info(f"Artikel '{article.title}' aufgrund niedrigen Relevanz-Scores ({article.relevance_score:.2f} < {relevance_threshold}) nicht für Newsletter ausgewählt.")

            except Exception as e:
                logger.error(f"Fehler bei der Evaluierung von Artikel '{article.title}': {e}", exc_info=True)
        
        logger.info(f"{len(final_approved_articles)} Artikel nach Evaluierung für den Newsletter ausgewählt.")
        return final_approved_articles

    def run_pipeline(self) -> Optional[str]:
        logger.info("Newsletter-Generierungspipeline gestartet.")
        start_time = datetime.now(timezone.utc)

        # 1. Daten sammeln
        raw_articles = self._fetch_all_raw_articles()
        if not raw_articles:
            logger.warning("Keine Roh-Artikel zum Verarbeiten gefunden. Breche Pipeline ab.")
            return None

        # 2. Daten initial verarbeiten
        initially_processed_articles = self._process_articles_initial(raw_articles)
        if not initially_processed_articles:
            logger.warning("Keine Artikel nach initialer LLM-Verarbeitung übrig. Breche Pipeline ab.")
            return None

        # 3. NEU: Artikel evaluieren und final auswählen
        final_articles_for_newsletter = self._evaluate_and_filter_articles(initially_processed_articles)
        if not final_articles_for_newsletter:
            logger.warning("Keine Artikel nach Evaluierung für den Newsletter ausgewählt. Breche Pipeline ab.")
            return None
        
        # 4. Newsletter-Datenstruktur erstellen und Sektionen zuordnen
        newsletter_sections_map: Dict[str, List[Any]] = {
            "Der Rund um Blick": [], "IT & AI": [], "Welt und Politik": [],
            "Wirtschaft": [], "Zürich Inside": [], "Kultur und Inspiration": []
        }

        for article in final_articles_for_newsletter:
            target_section = article.category if article.category in newsletter_sections_map else "Der Rund um Blick"
            newsletter_sections_map[target_section].append(article)

        final_sections: List[NewsletterSection] = []
        for title, items_list in newsletter_sections_map.items():
            if items_list:
                if all(isinstance(item, ProcessedArticle) for item in items_list):
                    items_list.sort(key=lambda x: (x.relevance_score or 0.0, x.published_at or datetime.min.replace(tzinfo=timezone.utc)), reverse=True)
                final_sections.append(NewsletterSection(title=title, items=items_list))
        
        if not final_sections:
            logger.warning("Keine Sektionen mit Inhalt für den Newsletter. Breche ab.")
            return None
            
        newsletter_content = NewsletterData(sections=final_sections)

        # 5. Newsletter komponieren (PDF)
        logger.info("Erstelle Newsletter-PDF...")
        composer = NewsletterComposer(newsletter_content)
        pdf_filename_base = f"Newsletter_{datetime.now(timezone.utc).strftime('%Y%m%d_%H%M%S')}"
        pdf_filepath = f"{pdf_filename_base}.pdf" # Speichert im aktuellen Verzeichnis
        
        try:
            abs_pdf_filepath = composer.generate_pdf(pdf_filepath)
            logger.info(f"PDF '{abs_pdf_filepath}' erfolgreich erstellt.")
        except Exception as e:
            logger.error(f"Fehler beim Erstellen des PDF: {e}", exc_info=True)
            return None

        # 6. Newsletter verteilen (z.B. Google Drive)
        # Hier ist der Code für den Fall, dass du es direkt in Python machst (optional):
        # drive_folder_id = get_env_variable("GOOGLE_DRIVE_FOLDER_ID")
        # service_account_file = get_env_variable("GOOGLE_APPLICATION_CREDENTIALS")

        # if drive_folder_id and service_account_file and os.path.exists(abs_pdf_filepath):
        #     logger.info(f"Lade '{abs_pdf_filepath}' auf Google Drive hoch...")
        #     uploader = GoogleDriveUploader(folder_id=drive_folder_id) # Initialisiert Service automatisch, wenn creds da
        #     if uploader.service: # Prüfen, ob Service initialisiert wurde
        #         drive_link = uploader.upload_file(local_filepath=abs_pdf_filepath, filename_on_drive=os.path.basename(abs_pdf_filepath))
                
        #         if drive_link:
        #             logger.info(f"Newsletter erfolgreich auf Google Drive hochgeladen: {drive_link}")
        #             # try:
        #             #     os.remove(abs_pdf_filepath) 
        #             #     logger.info(f"Lokale PDF-Datei '{abs_pdf_filepath}' gelöscht.")
        #             # except OSError as e_remove:
        #             #     logger.warning(f"Konnte lokale PDF-Datei '{abs_pdf_filepath}' nicht löschen: {e_remove}")
        #             pipeline_duration = datetime.now(timezone.utc) - start_time
        #             logger.info(f"Newsletter-Pipeline in {pipeline_duration} abgeschlossen.")
        #             return drive_link
        #         else:
        #             logger.error("Fehler beim Hochladen des Newsletters auf Google Drive.")
        #     else:
        #         logger.warning("Google Drive Service nicht initialisiert. Upload übersprungen.")
        # else:
        #     logger.warning(f"Google Drive Upload übersprungen. Bedingungen nicht erfüllt: folder_id='{drive_folder_id}', service_account_file='{service_account_file}', pdf_exists='{os.path.exists(abs_pdf_filepath) if abs_pdf_filepath else False}'.")
        
        pipeline_duration = datetime.now(timezone.utc) - start_time
        logger.info(f"Newsletter-Pipeline in {pipeline_duration} abgeschlossen. PDF lokal verfügbar unter: {abs_pdf_filepath}")
        return abs_pdf_filepath

# src/utils/config_loader.py
# Lädt Umgebungsvariablen und stellt Konfigurationen bereit.

import os
from dotenv import load_dotenv
import logging
from typing import Optional # Optional hinzugefügt

logger = logging.getLogger(__name__)

def load_env():
    """Lädt Umgebungsvariablen aus einer .env Datei im Projektwurzelverzeichnis."""
    project_root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
    dotenv_path = os.path.join(project_root, '.env')
    
    if os.path.exists(dotenv_path):
        load_dotenv(dotenv_path)
        logger.debug(f".env Datei geladen von: {dotenv_path}")
    else:
        logger.info(f".env Datei nicht gefunden unter: {dotenv_path}. Umgebungsvariablen müssen anderweitig gesetzt sein.")

def get_env_variable(variable_name: str, default: Optional[str] = None) -> Optional[str]:
    """Holt eine Umgebungsvariable. Gibt None zurück oder den Defaultwert, falls nicht gefunden."""
    value = os.getenv(variable_name, default)
    if value is None and default is None: 
        logger.debug(f"Umgebungsvariable '{variable_name}' nicht gefunden und kein Defaultwert angegeben.")
    return value

def get_api_key(key_name: str) -> str:
    """Holt einen API-Schlüssel. Löst einen Fehler aus, wenn nicht gefunden."""
    api_key = os.getenv(key_name)
    if not api_key:
        logger.critical(f"Kritischer Fehler: API-Schlüssel '{key_name}' nicht in den Umgebungsvariablen gefunden.")
        raise ValueError(f"API-Schlüssel '{key_name}' nicht in den Umgebungsvariablen gefunden. Bitte in .env setzen.")
    return api_key

# src/utils/logging_setup.py
# Konfiguriert das zentrale Logging für die Anwendung.

import logging
import sys
import os
from typing import Optional # Optional hinzugefügt

def setup_logging(log_level_str: str = "INFO", log_file: Optional[str] = None):
    """
    Konfiguriert das Logging-System.
    log_level_str: Logging-Level als String (z.B. "DEBUG", "INFO", "WARNING").
    log_file: Optionaler Pfad zu einer Log-Datei.
    """
    # Hole Default Log-Level aus Umgebungsvariable, falls gesetzt
    env_log_level = os.getenv("LOG_LEVEL", log_level_str).upper()
    numeric_level = getattr(logging, env_log_level, None)
    
    if not isinstance(numeric_level, int):
        print(f"Ungültiger Log-Level '{env_log_level}' aus Umgebung oder Default. Verwende INFO.")
        numeric_level = logging.INFO # Fallback auf INFO

    # Root-Logger konfigurieren
    # Es ist oft besser, Handler nicht immer wieder zu entfernen und hinzuzufügen,
    # sondern dies einmalig beim App-Start zu tun.
    # Hier stellen wir sicher, dass wir nur Handler hinzufügen, wenn noch keine da sind,
    # oder wenn wir spezifische Handler managen wollen.

    root_logger = logging.getLogger() # Hole den Root-Logger
    
    # Setze das Level für den Root-Logger. Alle Handler erben dieses Level, es sei denn, sie haben ein eigenes höheres Level.
    root_logger.setLevel(numeric_level) 

    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )

    # Konsolen-Handler hinzufügen, wenn noch keiner für stdout existiert
    if not any(isinstance(h, logging.StreamHandler) and h.stream == sys.stdout for h in root_logger.handlers):
        console_handler = logging.StreamHandler(sys.stdout)
        console_handler.setFormatter(formatter)
        # Setze das Level auch für den Handler, falls es feingranularer sein soll als das Root-Level
        console_handler.setLevel(numeric_level) 
        root_logger.addHandler(console_handler)
        logging.debug("Konsolen-Handler zum Root-Logger hinzugefügt.")
    else:
        logging.debug("Konsolen-Handler für stdout existiert bereits im Root-Logger.")


    # Hole Log-Datei-Pfad aus Umgebungsvariable, falls gesetzt, überschreibe Parameter
    env_log_file = os.getenv("LOG_FILE", log_file)
    if env_log_file:
        # Datei-Handler hinzufügen, wenn noch keiner für diesen Pfad existiert
        if not any(isinstance(h, logging.FileHandler) and h.baseFilename == os.path.abspath(env_log_file) for h in root_logger.handlers):
            try:
                log_dir = os.path.dirname(env_log_file)
                if log_dir and not os.path.exists(log_dir):
                    os.makedirs(log_dir)
                
                file_handler = logging.FileHandler(env_log_file, mode='a', encoding='utf-8')
                file_handler.setFormatter(formatter)
                file_handler.setLevel(numeric_level) # Setze Level auch für den Handler
                root_logger.addHandler(file_handler)
                logging.debug(f"Datei-Handler für '{env_log_file}' zum Root-Logger hinzugefügt.")
            except Exception as e:
                logging.error(f"Fehler beim Erstellen des Datei-Log-Handlers für '{env_log_file}': {e}", exc_info=True)
        else:
            logging.debug(f"Datei-Handler für '{env_log_file}' existiert bereits im Root-Logger.")


    # Test-Log-Nachricht, um zu zeigen, dass das Logging konfiguriert ist
    # Diese wird nur ausgegeben, wenn das effektive Level des Loggers INFO oder niedriger ist.
    logging.info(f"Logging konfiguriert mit effektivem Level {logging.getLevelName(root_logger.getEffectiveLevel())}" + (f" und potentieller Datei '{env_log_file}'" if env_log_file else ""))


# src/models/data_models.py
# Pydantic-Modelle zur Definition der Datenstrukturen.

from pydantic import BaseModel, HttpUrl, Field, field_validator, model_validator
from typing import Optional, List, Dict, Any, Union
from datetime import datetime, date, timezone

def ensure_timezone_aware(dt_value: Optional[datetime]) -> Optional[datetime]:
    """Stellt sicher, dass ein datetime-Objekt timezone-aware ist (UTC als Default)."""
    if dt_value is None:
        return None
    if isinstance(dt_value, str): # Zusätzliche Behandlung, falls String übergeben wird
        try:
            if dt_value.endswith('Z'):
                dt_value = datetime.fromisoformat(dt_value.replace('Z', '+00:00'))
            else:
                dt_value = datetime.fromisoformat(dt_value)
        except ValueError:
            # Fallback oder Fehlerbehandlung, wenn String nicht parsebar ist
            # Hier geben wir None zurück oder werfen einen Fehler, je nach Anforderung
            # Für pydantic ist es oft besser, einen Fehler zu werfen, damit die Validierung fehlschlägt
            raise ValueError(f"Ungültiges Datumsstring-Format für Konvertierung: {dt_value}")


    if dt_value.tzinfo is None or dt_value.tzinfo.utcoffset(dt_value) is None:
        return dt_value.replace(tzinfo=timezone.utc)
    return dt_value

class RawArticle(BaseModel):
    title: Optional[str] = None
    url: Optional[HttpUrl] = None
    description: Optional[str] = None
    content_snippet: Optional[str] = None 
    published_at: Optional[datetime] = None
    source_name: Optional[str] = None
    source_id: Optional[str] = None

    _ensure_published_at_tz_aware = field_validator('published_at', mode='before')(ensure_timezone_aware)


class ProcessedArticle(BaseModel):
    title: str
    url: Optional[HttpUrl] = None
    summary: str
    category: Optional[str] = "Unkategorisiert"
    relevance_score: Optional[float] = Field(default=0.0, ge=0.0, le=1.0) 
    source_name: Optional[str] = None
    published_at: Optional[datetime] = None
    # llm_processing_details kann jetzt komplexere Infos enthalten
    llm_processing_details: Optional[Dict[str, Any]] = Field(default_factory=dict)


    _ensure_published_at_tz_aware = field_validator('published_at', mode='before')(ensure_timezone_aware)

    # Beispiel für model_validator, falls benötigt
    # @model_validator(mode='after')
    # def check_consistency(self) -> 'ProcessedArticle':
    #     if self.relevance_score is not None and (self.relevance_score < 0 or self.relevance_score > 1):
    #         raise ValueError("Relevance score must be between 0 and 1")
    #     return self


class Event(BaseModel):
    summary: str 
    start_time: Optional[datetime] = None
    end_time: Optional[datetime] = None
    location: Optional[str] = None
    description: Optional[str] = None
    url: Optional[HttpUrl] = None
    source: str 

    _ensure_start_time_tz_aware = field_validator('start_time', mode='before')(ensure_timezone_aware)
    _ensure_end_time_tz_aware = field_validator('end_time', mode='before')(ensure_timezone_aware)


class Birthday(BaseModel):
    name: str
    date_month: int = Field(ge=1, le=12)
    date_day: int = Field(ge=1, le=31)
    original_date_info: Optional[str] = None 
    source: str 


class WeatherInfo(BaseModel):
    location: str
    temperature_celsius: float
    condition: str 
    humidity_percent: Optional[float] = None
    wind_speed_kmh: Optional[float] = None
    icon_url: Optional[HttpUrl] = None 
    forecast_snippet: Optional[str] = None 


class NewsletterSection(BaseModel):
    title: str 
    items: List[Union[ProcessedArticle, Event, Birthday, WeatherInfo, Dict[str, Any]]]


class NewsletterData(BaseModel):
    generation_date: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    title: str = "Dein personalisierter Newsletter"
    sections: List[NewsletterSection]

    _ensure_generation_date_tz_aware = field_validator('generation_date', mode='before')(ensure_timezone_aware)

# src/agents/data_fetchers/base_fetcher.py
# Abstrakte Basisklasse für alle Datenbeschaffer-Agenten.

from abc import ABC, abstractmethod
from typing import List, Any
import logging

logger = logging.getLogger(__name__) # Logger für dieses Modul

class BaseDataFetcher(ABC):
    """
    Abstrakte Basisklasse für Agenten, die Daten von externen Quellen abrufen.
    """
    def __init__(self, source_name: str):
        """
        Initialisiert den Fetcher mit einem Namen für die Quelle.
        Args:
            source_name (str): Ein identifizierbarer Name für die Datenquelle (z.B. "NewsAPI", "Tagesschau RSS").
        """
        self.source_name = source_name
        logger.info(f"Initialisiere Datenbeschaffer für Quelle: '{self.source_name}'.")

    @abstractmethod
    def fetch_data(self) -> List[Any]:
        """
        Abstrakte Methode zum Abrufen von Daten.
        Muss von abgeleiteten Klassen implementiert werden.

        Returns:
            List[Any]: Eine Liste von abgerufenen Datenelementen (z.B. RawArticle, Event).
                       Die genaue Struktur hängt vom spezifischen Fetcher ab.
        """
        pass

    def __repr__(self) -> str:
        return f"<{self.__class__.__name__}(source_name='{self.source_name}')>"

# src/agents/data_fetchers/newsapi_fetcher.py
# Datenbeschaffer für Nachrichten von NewsAPI.org.

import requests
from typing import List, Optional
from datetime import datetime, timedelta, timezone
from src.models.data_models import RawArticle
from src.utils.config_loader import get_api_key
from .base_fetcher import BaseDataFetcher
import logging
import json # Für das Parsen von Fehlermeldungen

logger = logging.getLogger(__name__)

class NewsAPIFetcher(BaseDataFetcher):
    """
    Ruft Nachrichtenartikel vom NewsAPI.org Dienst ab.
    Kann entweder den /v2/everything oder /v2/top-headlines Endpunkt verwenden.
    """
    def __init__(self, 
                 query: Optional[str] = None, 
                 country: Optional[str] = None,
                 category: Optional[str] = None,
                 sources: Optional[str] = None, # Komma-separierte Quellen-IDs
                 language: str = "de", 
                 days_ago: int = 1, 
                 endpoint: str = "everything", 
                 page_size: int = 20,
                 source_name_override: Optional[str] = None):
        
        effective_source_name = source_name_override if source_name_override else f"NewsAPI ({endpoint})"
        super().__init__(source_name=effective_source_name)
        
        self.api_key = get_api_key("NEWSAPI_API_KEY")
        self.base_url = "https://newsapi.org/v2/"
        self.query = query
        self.country = country
        self.category = category
        self.sources = sources
        self.language = language
        self.days_ago = days_ago
        self.endpoint = endpoint.lower()
        self.page_size = page_size

        if self.endpoint not in ["everything", "top-headlines"]:
            logger.error(f"Ungültiger NewsAPI Endpunkt '{self.endpoint}' für Quelle '{self.source_name}'.")
            raise ValueError("Ungültiger Endpunkt für NewsAPI. Muss 'everything' oder 'top-headlines' sein.")
        
        if self.endpoint == "top-headlines" and self.sources and (self.country or self.category):
            logger.warning(f"NewsAPI: Für /top-headlines wird 'sources' verwendet, 'country'/'category' werden ignoriert für Quelle '{self.source_name}'.")


    def _get_from_date(self) -> str:
        """Erstellt den Datumsstring für den 'from'-Parameter des 'everything'-Endpunkts."""
        # Nutze UTC für Konsistenz bei Datumsberechnungen
        return (datetime.now(timezone.utc) - timedelta(days=self.days_ago)).strftime("%Y-%m-%d")

    def fetch_data(self) -> List[RawArticle]:
        params = {
            "apiKey": self.api_key,
            "language": self.language,
            "pageSize": self.page_size,
        }

        if self.endpoint == "everything":
            if not self.query:
                logger.warning(f"Für den 'everything'-Endpunkt von '{self.source_name}' wird ein Suchbegriff ('query') dringend empfohlen.")
            params["q"] = self.query if self.query else "Aktuelles" 
            params["from"] = self._get_from_date()
            params["sortBy"] = "publishedAt"
        
        elif self.endpoint == "top-headlines":
            # Priorisierung: sources > country/category > q (für Top-Headlines)
            if self.sources:
                params["sources"] = self.sources
            elif self.country:
                params["country"] = self.country
                if self.category: 
                    params["category"] = self.category
            elif self.category:
                params["category"] = self.category
            if self.query: # 'q' kann auch mit /top-headlines verwendet werden
                params["q"] = self.query
            if not (self.sources or self.country or self.category or self.query):
                logger.error(f"Für NewsAPI /top-headlines muss mindestens einer der Parameter 'sources', 'country', 'category' oder 'q' gesetzt sein für Quelle '{self.source_name}'.")
                return [] # Leere Liste, da die Anfrage so nicht sinnvoll ist
        
        url = f"{self.base_url}{self.endpoint}"
        params_to_log = {k: v for k, v in params.items() if k != 'apiKey'}
        logger.info(f"Frage NewsAPI ({self.source_name}) ab: {url} mit Parametern: {params_to_log}")
        
        raw_articles: List[RawArticle] = []
        try:
            response = requests.get(url, params=params, timeout=20)
            
            # Logge Status und einen Teil der Antwort immer, um Debugging zu erleichtern
            logger.debug(f"NewsAPI ({self.source_name}) Status Code: {response.status_code}")
            # Logge nur einen Teil der Antwort, um Logs nicht zu überfluten
            # response_snippet = response.text[:500] + "..." if response.text else "Keine Antwort"
            # logger.debug(f"NewsAPI ({self.source_name}) Antwort Snippet: {response_snippet}")

            response.raise_for_status() 
            data = response.json()

            if data.get("status") == "error":
                logger.error(f"NewsAPI Fehler ({self.source_name}): Code '{data.get('code')}', Message '{data.get('message')}'")
                return []

            articles_data = data.get("articles", [])
            logger.info(f"NewsAPI ({self.source_name}) lieferte {data.get('totalResults', 0)} Gesamtartikel, {len(articles_data)} im aktuellen Batch.")

            for article_data in articles_data:
                try:
                    # publishedAt Parsing verbessert
                    published_at_str = article_data.get("publishedAt")
                    published_dt: Optional[datetime] = None
                    if published_at_str:
                        try:
                            # Stellt sicher, dass das Datum als timezone-aware UTC geparsed wird
                            # datetime.fromisoformat kann 'Z' direkt parsen seit Python 3.11
                            # Für ältere Versionen ist .replace('Z', '+00:00') nötig.
                            if published_at_str.endswith('Z'):
                                 published_dt = datetime.fromisoformat(published_at_str[:-1] + '+00:00')
                            else: # Versuche, es direkt zu parsen
                                temp_dt = datetime.fromisoformat(published_at_str)
                                # Mache es timezone-aware, falls es das nicht ist (nehme UTC an)
                                if temp_dt.tzinfo is None or temp_dt.tzinfo.utcoffset(temp_dt) is None:
                                    published_dt = temp_dt.replace(tzinfo=timezone.utc)
                                else:
                                    published_dt = temp_dt # Bereits timezone-aware
                        except ValueError:
                            logger.warning(f"Konnte publishedAt '{published_at_str}' nicht parsen für Artikel von '{self.source_name}': {article_data.get('title')}")
                    
                    raw_articles.append(
                        RawArticle(
                            title=article_data.get("title"),
                            url=str(article_data.get("url")) if article_data.get("url") else None, # Stelle sicher, dass es ein String ist für Pydantic HttpUrl
                            description=article_data.get("description"),
                            content_snippet=article_data.get("content"),
                            published_at=published_dt,
                            source_name=article_data.get("source", {}).get("name", self.source_name),
                            source_id=article_data.get("source", {}).get("id")
                        )
                    )
                except Exception as e_article_parse:
                    logger.warning(f"Überspringe Artikel von '{self.source_name}' aufgrund eines Parsing-Fehlers: '{article_data.get('title', 'Unbekannt')}' - Fehler: {e_article_parse}", exc_info=False)
            
            logger.info(f"{len(raw_articles)} Artikel erfolgreich von '{self.source_name}' abgerufen und geparsed.")

        except requests.exceptions.HTTPError as e_http:
            error_message = f"HTTP-Fehler ({e_http.response.status_code}) beim Abrufen von '{self.source_name}'"
            try:
                error_details = e_http.response.json()
                error_message += f": {error_details.get('code')} - {error_details.get('message')}"
            except json.JSONDecodeError:
                error_message += f". Rohantwort: {e_http.response.text[:200]}" # Snippet der Rohantwort
            logger.error(error_message, exc_info=False) # exc_info=False um Log nicht mit vollem Traceback zu überfluten
        except requests.exceptions.RequestException as e_req:
            logger.error(f"Netzwerkfehler beim Abrufen von Daten von '{self.source_name}': {e_req}", exc_info=True)
        except Exception as e_general:
            logger.error(f"Unerwarteter Fehler beim Verarbeiten von Daten von '{self.source_name}': {e_general}", exc_info=True)
        
        return raw_articles

# src/agents/llm_processors/base_processor.py
# Abstrakte Basisklasse für alle LLM-basierten Verarbeitungs-Agenten.

from abc import ABC, abstractmethod
from typing import Any, Optional
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.language_models.chat_models import BaseChatModel
from src.utils.config_loader import get_api_key, get_env_variable
import logging

logger = logging.getLogger(__name__)

class BaseLLMProcessor(ABC):
    """
    Abstrakte Basisklasse für Agenten, die Daten mithilfe eines LLM verarbeiten.
    Verwendet standardmäßig Google Gemini.
    """
    def __init__(self, 
                 model_name: Optional[str] = None, 
                 temperature: float = 0.3,
                 llm_provider: str = "gemini"): 
        self.llm_provider = llm_provider.lower()
        self.model_name = model_name
        self.temperature = temperature
        self.llm: Optional[BaseChatModel] = None

        try:
            if self.llm_provider == "gemini":
                gemini_api_key = get_api_key("GOOGLE_API_KEY") 
                # Hole Default-Modellnamen aus .env, falls vorhanden, sonst Fallback
                default_gemini_model = get_env_variable("GEMINI_DEFAULT_MODEL", "gemini-1.5-flash-latest")
                self.model_name = model_name if model_name else default_gemini_model
                
                self.llm = ChatGoogleGenerativeAI(
                    model=self.model_name,
                    google_api_key=gemini_api_key,
                    temperature=self.temperature,
                    convert_system_message_to_human=True 
                )
                logger.info(f"Initialisiere Gemini LLM Processor mit Modell: {self.model_name}, Temperatur: {self.temperature}.")
            else:
                logger.error(f"Nicht unterstützter LLM-Provider: {self.llm_provider}")
                raise ValueError(f"Nicht unterstützter LLM-Provider: {self.llm_provider}")
        except Exception as e:
            logger.critical(f"Fehler bei der Initialisierung des LLM-Clients für Provider '{self.llm_provider}': {e}", exc_info=True)
            raise 

    @abstractmethod
    def process(self, data: Any, **kwargs) -> Any:
        if self.llm is None:
            logger.error("LLM wurde nicht korrekt initialisiert. Verarbeitung nicht möglich.")
            raise RuntimeError("LLM nicht initialisiert. Verarbeitung abgebrochen.")
        pass

    def __repr__(self) -> str:
        return f"<{self.__class__.__name__}(model='{self.model_name}', provider='{self.llm_provider}')>"

# src/agents/llm_processors/summarizer_agent.py
# LLM-Agent zum Zusammenfassen von Texten (z.B. Artikel).

from typing import List, Union, Optional # Optional hinzugefügt
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from src.models.data_models import RawArticle, ProcessedArticle, Event 
from .base_processor import BaseLLMProcessor
import logging

logger = logging.getLogger(__name__)

class SummarizerAgent(BaseLLMProcessor):
    def __init__(self, 
                 model_name: Optional[str] = None, 
                 temperature: float = 0.2): 
        super().__init__(model_name=model_name, temperature=temperature)
        
        self.prompt_template = ChatPromptTemplate.from_messages([
            ("human", 
             """
Du bist ein Experte im Verfassen prägnanter Nachrichten-Zusammenfassungen für einen anspruchsvollen Leser.
Bitte fasse den folgenden Text für einen Newsletter zusammen. Die Zusammenfassung sollte die Kernbotschaft in 2-4 prägnanten Sätzen wiedergeben.
Konzentriere dich auf die wichtigsten Fakten und Implikationen. Vermeide Füllwörter.
Gib NUR die reine Zusammenfassung zurück, ohne zusätzliche Einleitungen, Höflichkeitsfloskeln oder Kommentare.

ARTIKELTITEL: {title}
ARTIKELBESCHREIBUNG (falls vorhanden): {description}
ARTIKELINHALT (Auszug, falls vorhanden, sonst leer): {content_snippet}

ZUSAMMENFASSUNG:""")
        ])
        
        self.chain = self.prompt_template | self.llm | StrOutputParser()
        logger.info("SummarizerAgent Kette initialisiert.")

    def _get_text_for_summarization(self, item: Union[RawArticle, Event]) -> str:
        text_candidates = []
        if isinstance(item, RawArticle):
            if item.content_snippet and len(item.content_snippet.strip()) > 50: # Bevorzuge längeren Inhalt
                text_candidates.append(item.content_snippet.strip())
            if item.description and len(item.description.strip()) > 20:
                text_candidates.append(item.description.strip())
            if item.title: # Titel als letzter Fallback, falls alles andere fehlt
                text_candidates.append(item.title.strip())
        elif isinstance(item, Event):
            if item.description and len(item.description.strip()) > 20:
                text_candidates.append(item.description.strip())
            if item.summary: # summary ist Titel des Events
                 text_candidates.append(item.summary.strip())
        
        # Wähle den längsten, sinnvollen Kandidaten
        if text_candidates:
            return max(text_candidates, key=len)
        return ""


    def summarize_article(self, article: RawArticle) -> str:
        logger.debug(f"Zusammenfassung für Artikel angefordert: '{article.title or 'Unbekannter Titel'}'")
        
        text_to_summarize = self._get_text_for_summarization(article)
        
        if not text_to_summarize:
            logger.warning(f"Kein ausreichender Inhalt zum Zusammenfassen für Artikel: {article.title or 'Unbekannter Titel'}")
            return article.description or article.title or "Keine Zusammenfassung verfügbar (unzureichender Inhalt)."

        try:
            max_input_chars = 4000 # Etwas erhöht, aber immer noch vorsichtig
            
            summary_result = self.chain.invoke({
                "title": article.title or "Kein Titel",
                "description": article.description or "", # description kann auch leer sein
                "content_snippet": text_to_summarize[:max_input_chars] 
            })
            
            logger.debug(f"Zusammenfassung für '{article.title}' erstellt.")
            return summary_result.strip() if summary_result else "Zusammenfassung konnte nicht erstellt werden."
        except Exception as e:
            logger.error(f"Fehler beim Zusammenfassen des Artikels '{article.title}': {e}", exc_info=True)
            return article.description or article.title or "Zusammenfassung fehlgeschlagen (LLM-Fehler)."

    def process(self, articles: List[RawArticle]) -> List[ProcessedArticle]:
        if self.llm is None: 
             logger.error("LLM nicht initialisiert im SummarizerAgent. Breche Verarbeitung ab.")
             return []
             
        logger.info(f"Starte Batch-Zusammenfassung für {len(articles)} Artikel.")
        processed_articles_list: List[ProcessedArticle] = []
        
        for article_item in articles:
            summary_text = self.summarize_article(article_item)
            processed_articles_list.append(
                ProcessedArticle(
                    title=article_item.title or "Unbekannter Titel",
                    url=article_item.url,
                    summary=summary_text,
                    source_name=article_item.source_name,
                    published_at=article_item.published_at
                )
            )
        logger.info(f"Batch-Zusammenfassung für {len(processed_articles_list)} Artikel abgeschlossen.")
        return processed_articles_list

# src/agents/llm_processors/categorizer_agent.py
# LLM-Agent zum Kategorisieren von Texten (z.B. Artikel).

from typing import List, Optional, Union, Dict # Union, Dict hinzugefügt
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser # Behalte JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel as LangchainBaseModel, Field as LangchainField # Field hinzugefügt
from src.models.data_models import RawArticle 
from .base_processor import BaseLLMProcessor
import logging
import json 

logger = logging.getLogger(__name__)

class CategorizationResponse(LangchainBaseModel):
    category: str = LangchainField(description="Die am besten passende Kategorie für den Artikel aus der vorgegebenen Liste.")
    # Optional: confidence und reasoning können beibehalten oder entfernt werden, je nach Bedarf.
    # confidence: Optional[float] = LangchainField(description="Konfidenzwert der Kategorisierung (0.0 bis 1.0).", default=None)
    # reasoning: Optional[str] = LangchainField(description="Kurze Begründung für die gewählte Kategorie.", default=None)


class CategorizerAgent(BaseLLMProcessor):
    def __init__(self, 
                 categories: List[str],
                 model_name: Optional[str] = None,
                 temperature: float = 0.1): 
        super().__init__(model_name=model_name, temperature=temperature)
        
        if not categories:
            logger.error("CategorizerAgent erfordert eine Liste von Kategorien bei der Initialisierung.")
            raise ValueError("Es müssen Kategorien für den CategorizerAgent bereitgestellt werden.")
        self.categories_list = categories # Speichere als Liste für die Validierung
        self.categories_str = ", ".join(f"'{cat}'" for cat in categories)

        self.prompt_template = ChatPromptTemplate.from_template( # Vereinfacht zu from_template
             """
Du bist ein Experte für die thematische Kategorisierung von Nachrichtenartikeln.
Deine Aufgabe ist es, den folgenden Artikel EINER der vorgegebenen Kategorien zuzuordnen.
Antworte ausschließlich im JSON-Format. Das JSON-Objekt muss einen Schlüssel "category" enthalten, dessen Wert eine der unten genannten Kategorien ist.

Vorgegebene Kategorien: [{available_categories}]

Wähle nur eine einzige, die relevanteste Kategorie.

ARTIKELTITEL: {title}
ARTIKELZUSAMMENFASSUNG oder TEXTAUSZUG: {text_snippet}

JSON-ANTWORT (nur das JSON-Objekt, ohne Markdown):
"""
        )
        
        self.output_parser = JsonOutputParser(pydantic_object=CategorizationResponse)
        self.chain = self.prompt_template | self.llm | self.output_parser
        logger.info(f"CategorizerAgent Kette initialisiert mit Kategorien: {self.categories_str}.")

    def _get_text_for_categorization(self, item: RawArticle, summary: Optional[str] = None) -> str:
        text_candidates = []
        if summary and len(summary.strip()) > 20: # Bevorzuge Zusammenfassung, wenn vorhanden
            text_candidates.append(summary.strip())
        
        if isinstance(item, RawArticle):
            # Füge Beschreibung hinzu, wenn keine gute Zusammenfassung da ist oder sie kurz ist
            if item.description and len(item.description.strip()) > 20:
                text_candidates.append(item.description.strip())
            # Titel als letzter Ausweg oder zur Ergänzung
            if item.title:
                text_candidates.append(item.title.strip())
        
        if text_candidates:
            # Kombiniere Titel und längsten anderen Text für mehr Kontext, aber halte es kurz
            title_part = item.title if item.title else ""
            best_other_text = max((tc for tc in text_candidates if tc != title_part), key=len, default="")
            # return f"{title_part} - {best_other_text}".strip(" -") if best_other_text else title_part
            # Für reine Kategorisierung ist der längste verfügbare Text oft am besten:
            return max(text_candidates, key=len)

        return ""


    def categorize_article(self, article: RawArticle, summary_for_categorization: Optional[str] = None) -> str:
        logger.debug(f"Kategorisierung für Artikel angefordert: '{article.title or 'Unbekannter Titel'}'")
        
        text_to_categorize = self._get_text_for_categorization(article, summary_for_categorization)
        
        if not text_to_categorize:
            logger.warning(f"Kein ausreichender Inhalt zum Kategorisieren für Artikel: {article.title or 'Unbekannter Titel'}")
            return "Unkategorisiert" 

        try:
            max_input_chars = 2000 
            
            # Das Ergebnis des JsonOutputParser ist bereits ein Dictionary (oder das Pydantic-Objekt)
            response_dict: Dict = self.chain.invoke({
                "available_categories": self.categories_str,
                "title": article.title or "Kein Titel",
                "text_snippet": text_to_categorize[:max_input_chars] 
            })
            
            category = response_dict.get("category", "Unkategorisiert")

            # Stelle sicher, dass die zurückgegebene Kategorie eine der erlaubten ist
            if category not in self.categories_list and category != "Unkategorisiert":
                logger.warning(f"LLM gab eine ungültige Kategorie '{category}' zurück für Artikel '{article.title}'. Setze auf 'Unkategorisiert'.")
                category = "Unkategorisiert"

            logger.debug(f"Artikel '{article.title}' kategorisiert als: '{category}'.")
            return category

        except Exception as e:
            logger.error(f"Fehler beim Kategorisieren des Artikels '{article.title}': {e}", exc_info=True)
            return "Fehler bei Kategorisierung" 

    def process(self, articles_with_summaries: List[Dict[str, Union[RawArticle, str]]]) -> List[Dict[str, Union[RawArticle, str, str]]]:
        if self.llm is None:
             logger.error("LLM nicht initialisiert im CategorizerAgent. Breche Verarbeitung ab.")
             return []
             
        logger.info(f"Starte Batch-Kategorisierung für {len(articles_with_summaries)} Artikel.")
        categorized_items_list = []
        
        for item_data_dict in articles_with_summaries:
            raw_article_obj = item_data_dict.get('raw_article')
            summary_text = item_data_dict.get('summary')
            if not isinstance(raw_article_obj, RawArticle):
                logger.warning(f"Ungültiges Item für Kategorisierung übersprungen: {raw_article_obj}")
                continue

            category_name = self.categorize_article(raw_article_obj, summary_text)
            item_data_dict['category'] = category_name # Füge Kategorie zum existierenden Dict hinzu
            categorized_items_list.append(item_data_dict)
            
        logger.info(f"Batch-Kategorisierung für {len(categorized_items_list)} Artikel abgeschlossen.")
        return categorized_items_list

# src/agents/llm_processors/evaluator_agent.py
# NEUE DATEI: LLM-Agent zur Evaluierung von Artikeln (z.B. Zielgruppenrelevanz)

from typing import List, Dict, Optional, Union
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel as LangchainBaseModel, Field as LangchainField
from src.models.data_models import ProcessedArticle # Arbeitet mit bereits zusammengefassten Artikeln
from .base_processor import BaseLLMProcessor
import logging
import json

logger = logging.getLogger(__name__)

# Pydantic-Modell für die erwartete JSON-Ausgabe der Evaluierung
class ArticleEvaluationResponse(LangchainBaseModel):
    relevance_score: float = LangchainField(description="Ein Score von 0.0 (irrelevant) bis 1.0 (hoch relevant) für die Zielgruppe.", ge=0.0, le=1.0)
    is_recommended: bool = LangchainField(description="True, wenn der Artikel für den Newsletter empfohlen wird, sonst False.")
    feedback_summary: Optional[str] = LangchainField(description="Kurzes Feedback oder Begründung für die Bewertung.", default=None)
    identified_keywords: Optional[List[str]] = LangchainField(description="Schlüsselwörter im Artikel, die zur Relevanz beitragen.", default_factory=list)

class AudienceAlignmentAgent(BaseLLMProcessor):
    """
    Ein LLM-Agent, der bewertet, wie gut ein Artikel zur Zielgruppe des Newsletters passt.
    """
    def __init__(self, 
                 user_profile: str, # Beschreibung der Zielgruppe/Nutzerinteressen
                 model_name: Optional[str] = None,
                 temperature: float = 0.2):
        super().__init__(model_name=model_name, temperature=temperature)
        
        if not user_profile or len(user_profile.strip()) < 20: # Mindestlänge für ein sinnvolles Profil
            logger.error("AudienceAlignmentAgent erfordert ein aussagekräftiges Nutzerprofil.")
            raise ValueError("Ein detailliertes Nutzerprofil ist für den AudienceAlignmentAgent erforderlich.")
        self.user_profile = user_profile

        self.prompt_template = ChatPromptTemplate.from_template(
             """
Du bist ein kritischer Redakteur, der die Relevanz von Nachrichtenartikeln für eine spezifische Zielgruppe bewertet.
Deine Aufgabe ist es, den folgenden Artikel im Kontext des gegebenen Nutzerprofils zu analysieren.
Antworte ausschließlich im JSON-Format gemäß der folgenden Struktur:
{{
  "relevance_score": float (0.0 bis 1.0, wobei 1.0 perfekt passt),
  "is_recommended": boolean (true, wenn der Artikel für den Newsletter basierend auf dem Profil empfohlen wird),
  "feedback_summary": string (kurze Begründung für deine Bewertung und Empfehlung, max. 2 Sätze),
  "identified_keywords": list[string] (3-5 Schlüsselwörter aus dem Artikel, die für die Relevanzentscheidung wichtig waren)
}}

NUTZERPROFIL / ZIELGRUPPE:
---
{user_profile_description}
---

ZU BEWERTENDER ARTIKEL:
Titel: {article_title}
Zusammenfassung: {article_summary}
Quelle: {article_source}
Kategorie (falls bekannt): {article_category}

JSON-BEWERTUNG:
"""
        )
        
        self.output_parser = JsonOutputParser(pydantic_object=ArticleEvaluationResponse)
        self.chain = self.prompt_template | self.llm | self.output_parser
        logger.info(f"AudienceAlignmentAgent Kette initialisiert.")

    def evaluate_article(self, article: ProcessedArticle) -> Dict:
        """Bewertet einen einzelnen ProcessedArticle auf Zielgruppenrelevanz."""
        logger.debug(f"Bewerte Artikel auf Zielgruppenrelevanz: '{article.title}'")
        
        if not article.summary or len(article.summary.strip()) < 10:
            logger.warning(f"Artikel '{article.title}' hat keine ausreichende Zusammenfassung für die Bewertung. Gebe niedrigen Score.")
            return {"relevance_score": 0.1, "is_recommended": False, "feedback_summary": "Keine ausreichende Zusammenfassung für eine Bewertung.", "identified_keywords": []}

        try:
            max_summary_chars = 1000 # Nur die Zusammenfassung verwenden
            
            response_data: Union[Dict, ArticleEvaluationResponse] = self.chain.invoke({
                "user_profile_description": self.user_profile,
                "article_title": article.title,
                "article_summary": article.summary[:max_summary_chars],
                "article_source": article.source_name or "Unbekannt",
                "article_category": article.category or "Unkategorisiert"
            })
            
            # Konvertiere Pydantic-Modell zu Dict, falls es nicht schon eines ist
            if isinstance(response_data, LangchainBaseModel):
                evaluation_dict = response_data.dict()
            else:
                evaluation_dict = response_data # Bereits ein Dict

            logger.debug(f"Evaluierung für '{article.title}': Score={evaluation_dict.get('relevance_score')}, Empfohlen={evaluation_dict.get('is_recommended')}")
            return evaluation_dict

        except Exception as e:
            logger.error(f"Fehler bei der Evaluierung des Artikels '{article.title}': {e}", exc_info=True)
            # Fallback-Bewertung bei Fehler
            return {"relevance_score": 0.0, "is_recommended": False, "feedback_summary": f"Fehler bei der Evaluierung: {str(e)[:100]}", "identified_keywords": []}

    def process(self, articles: List[ProcessedArticle]) -> List[Dict]: # Gibt eine Liste von Evaluations-Dicts zurück
        """Verarbeitet eine Liste von ProcessedArticle-Objekten und gibt für jeden eine Bewertung zurück."""
        if self.llm is None:
             logger.error("LLM nicht initialisiert im AudienceAlignmentAgent. Breche Verarbeitung ab.")
             return []
             
        logger.info(f"Starte Batch-Evaluierung für {len(articles)} Artikel.")
        evaluations = []
        for article_item in articles:
            evaluation = self.evaluate_article(article_item)
            evaluations.append(evaluation) # Füge das gesamte Evaluations-Dict hinzu
            
        logger.info(f"Batch-Evaluierung für {len(evaluations)} Artikel abgeschlossen.")
        return evaluations

# src/agents/newsletter_composer.py
# Erstellt den Newsletter (z.B. als PDF).

from typing import List, Any, Dict
from src.models.data_models import NewsletterData, NewsletterSection, ProcessedArticle, Event, Birthday, WeatherInfo
from datetime import datetime, timezone
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch, cm
from reportlab.lib.enums import TA_JUSTIFY, TA_LEFT, TA_CENTER, TA_RIGHT
from reportlab.lib import colors
from reportlab.pdfbase.pdfmetrics import registerFont
from reportlab.pdfbase.ttfonts import TTFont
import logging
import os

logger = logging.getLogger(__name__)

class NewsletterComposer:
    """
    Erstellt den Newsletter aus den aufbereiteten Daten, typischerweise als PDF.
    """
    def __init__(self, newsletter_data: NewsletterData):
        self.newsletter_data = newsletter_data
        self.styles = getSampleStyleSheet()
        self._register_fonts() 
        self._define_custom_styles()
        logger.info("Initialisiere Newsletter Composer.")

    def _register_fonts(self):
        # Hier könnten benutzerdefinierte Schriftarten registriert werden (siehe vorheriges Beispiel)
        pass 

    def _define_custom_styles(self):
        self.styles.add(ParagraphStyle(name='H1Centered', parent=self.styles['h1'], alignment=TA_CENTER, spaceAfter=0.3*cm, fontSize=18))
        self.styles.add(ParagraphStyle(name='H2Left', parent=self.styles['h2'], alignment=TA_LEFT, spaceBefore=0.6*cm, spaceAfter=0.2*cm, textColor=colors.HexColor("#003366"), fontSize=16)) # Dunkelblau
        self.styles.add(ParagraphStyle(name='H3Article', parent=self.styles['h3'], alignment=TA_LEFT, spaceAfter=0.1*cm, textColor=colors.HexColor("#333333"), fontSize=14)) # Dunkelgrau
        self.styles.add(ParagraphStyle(name='NormalJustified', parent=self.styles['Normal'], alignment=TA_JUSTIFY, spaceAfter=0.2*cm, leading=14, fontSize=10)) # Zeilenabstand, Schriftgröße
        self.styles.add(ParagraphStyle(name='ItalicSmall', parent=self.styles['Italic'], fontSize=8, spaceBefore=0.05*cm, spaceAfter=0.1*cm, textColor=colors.dimgrey))
        self.styles.add(ParagraphStyle(name='LinkStyle', parent=self.styles['Normal'], textColor=colors.blue, fontSize=9)) 
        self.styles.add(ParagraphStyle(name='Footer', parent=self.styles['Normal'], alignment=TA_CENTER, fontSize=8, textColor=colors.grey))
        self.styles.add(ParagraphStyle(name='RelevanceScore', parent=self.styles['Normal'], fontSize=8, textColor=colors.darkgreen, alignment=TA_RIGHT, spaceBefore=0.1*cm))


    def _add_header_footer(self, canvas, doc):
        """Fügt Kopf- und Fußzeile zu jeder Seite hinzu."""
        canvas.saveState()
        # Fußzeile
        footer_text = f"Seite {doc.page} - {self.newsletter_data.generation_date.strftime('%d.%m.%Y')}"
        canvas.setFont('Times-Roman', 8)
        canvas.setFillColor(colors.grey)
        canvas.drawCentredString(doc.width/2.0 + doc.leftMargin, 0.75 * inch, footer_text)
        
        # Optional: Kopfzeile
        # header_text = "Dein täglicher Newsletter"
        # canvas.setFont('Times-Roman', 9)
        # canvas.setFillColor(colors.darkgrey)
        # canvas.drawString(doc.leftMargin, doc.height + doc.topMargin - 0.5 * inch, header_text)
        canvas.restoreState()

    def generate_pdf(self, filename: str = "newsletter.pdf") -> str:
        logger.info(f"Starte PDF-Generierung: '{filename}'")
        
        output_dir = os.path.dirname(filename)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"Ausgabeverzeichnis '{output_dir}' erstellt.")

        doc = SimpleDocTemplate(filename,
                                rightMargin=1.5*cm, leftMargin=1.5*cm,
                                topMargin=1.8*cm, bottomMargin=1.8*cm) # Margins angepasst
        
        story: List[Any] = []

        newsletter_title_text = self.newsletter_data.title or "Dein personalisierter Newsletter"
        story.append(Paragraph(newsletter_title_text, self.styles['H1Centered']))
        
        gen_date_str = self.newsletter_data.generation_date.strftime('%A, %d. %B %Y, %H:%M Uhr (UTC)')
        story.append(Paragraph(f"<i>Erstellt am: {gen_date_str}</i>", self.styles['ItalicSmall']))
        story.append(Spacer(1, 0.6*cm))

        for section_idx, section in enumerate(self.newsletter_data.sections):
            if not section.items: 
                logger.debug(f"Überspringe leere Sektion: '{section.title}'")
                continue

            # Optional: Seitenumbruch vor jeder neuen Hauptsektion, außer der ersten
            # if section_idx > 0:
            #     story.append(PageBreak())

            story.append(Paragraph(section.title, self.styles['H2Left']))
            story.append(Spacer(1, 0.1*cm))
            
            for item in section.items:
                if isinstance(item, ProcessedArticle):
                    story.append(Paragraph(f"<b>{item.title or 'Kein Titel'}</b>", self.styles['H3Article']))
                    
                    meta_info_parts = []
                    if item.source_name:
                        meta_info_parts.append(f"Quelle: {item.source_name}")
                    if item.published_at:
                        meta_info_parts.append(item.published_at.strftime('%d.%m.%y %H:%M'))
                    if meta_info_parts:
                        story.append(Paragraph(f"<i>{' | '.join(meta_info_parts)}</i>", self.styles['ItalicSmall']))

                    story.append(Paragraph(item.summary or "Keine Zusammenfassung.", self.styles['NormalJustified']))
                    
                    if item.relevance_score is not None: # Relevanz-Score anzeigen
                         story.append(Paragraph(f"Relevanz: {item.relevance_score*100:.0f}%", self.styles['RelevanceScore']))

                    if item.url:
                        story.append(Paragraph(f"<a href='{str(item.url)}' color='blue'><u>Artikel online lesen</u></a>", self.styles['LinkStyle']))
                    story.append(Spacer(1, 0.4*cm)) # Etwas mehr Abstand

                # ... (Behandlung für Event, Birthday, WeatherInfo wie zuvor, ggf. auch mit Anpassungen) ...
                
                else: 
                    story.append(Paragraph(f"Unbekannter Eintrag: {str(item)[:100]}...", self.styles['Normal']))
                    story.append(Spacer(1, 0.3*cm))
            story.append(Spacer(1, 0.6*cm)) 

        try:
            doc.build(story, onFirstPage=self._add_header_footer, onLaterPages=self._add_header_footer)
            abs_filepath = os.path.abspath(filename)
            logger.info(f"PDF '{abs_filepath}' erfolgreich erstellt.")
            return abs_filepath 
        except Exception as e:
            logger.error(f"Fehler beim Erstellen des PDF: {e}", exc_info=True)
            raise

# src/agents/distributors/google_drive_uploader.py
# Agent zum Hochladen von Dateien auf Google Drive.

from typing import Optional
# Die Google Client Library muss installiert sein: pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
from google.oauth2.service_account import Credentials 
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError
from src.utils.config_loader import get_env_variable 
import logging
import os
import json # Für Fehlerdetails

logger = logging.getLogger(__name__)

class GoogleDriveUploader:
    """
    Lädt Dateien auf Google Drive hoch.
    Verwendet ein Service Account für die Authentifizierung.
    """
    def __init__(self, folder_id: Optional[str] = None):
        self.folder_id = folder_id
        self.service = None
        self._initialize_service() # Rufe Initialisierung im Konstruktor auf
        if self.service:
             logger.info(f"Google Drive Uploader initialisiert. Zielordner-ID: '{self.folder_id if self.folder_id else 'Root des Service Accounts'}'.")
        # Keine Fehlermeldung hier, wenn _initialize_service fehlschlägt, da es dort schon loggt.

    def _initialize_service(self):
        """Initialisiert den Google Drive API Service."""
        try:
            credentials_path = get_env_variable("GOOGLE_APPLICATION_CREDENTIALS")
            if not credentials_path:
                logger.warning("Umgebungsvariable 'GOOGLE_APPLICATION_CREDENTIALS' nicht gesetzt. Google Drive Uploads sind deaktiviert.")
                return

            if not os.path.exists(credentials_path):
                logger.error(f"Service Account Schlüsseldatei nicht gefunden unter: {credentials_path}. Google Drive Uploads sind deaktiviert.")
                return

            scopes = ['https://www.googleapis.com/auth/drive.file'] 
            
            creds = Credentials.from_service_account_file(credentials_path, scopes=scopes)
            # cache_discovery=False kann bei Problemen in manchen Umgebungen helfen (z.B. Serverless Functions)
            self.service = build('drive', 'v3', credentials=creds, cache_discovery=False) 
            logger.info("Google Drive Service erfolgreich initialisiert.")

        except Exception as e:
            logger.error(f"Fehler bei der Initialisierung des Google Drive Service: {e}", exc_info=True)
            self.service = None 

    def upload_file(self, local_filepath: str, filename_on_drive: str, mimetype: str = 'application/pdf') -> Optional[str]:
        if not self.service:
            logger.error("Google Drive Service nicht initialisiert. Datei-Upload abgebrochen.")
            return None
        
        if not os.path.exists(local_filepath):
            logger.error(f"Lokale Datei nicht gefunden: '{local_filepath}'. Upload abgebrochen.")
            return None

        try:
            file_metadata = {'name': filename_on_drive}
            if self.folder_id:
                file_metadata['parents'] = [self.folder_id]
            
            media = MediaFileUpload(local_filepath, mimetype=mimetype, resumable=True)
            
            logger.info(f"Starte Upload von '{local_filepath}' als '{filename_on_drive}' nach Google Drive...")
            
            request = self.service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id, name, webViewLink' 
            )
            
            file_resource = request.execute()
            
            file_id = file_resource.get('id')
            file_name = file_resource.get('name')
            web_view_link = file_resource.get('webViewLink')

            logger.info(f"Datei '{file_name}' erfolgreich auf Google Drive hochgeladen. ID: {file_id}, Link: {web_view_link}")
            return web_view_link

        except HttpError as error:
            error_reason = "Unbekannter Grund"
            error_details_str = ""
            try:
                # Versuche, Details aus dem Fehlercontent zu extrahieren
                error_content = error.content.decode('utf-8')
                error_details = json.loads(error_content)
                error_reason = error_details.get('error', {}).get('message', error._get_reason())
                error_details_str = f" Fehlerdetails: {error_details}"
            except Exception: # Falls das Parsen fehlschlägt
                error_reason = error._get_reason()
                error_details_str = f" Roh-Fehlercontent: {error.content.decode() if error.content else 'Kein Content'}"

            logger.error(f"Ein HTTP-Fehler ist beim Google Drive Upload aufgetreten: {error.resp.status} - {error_reason}.{error_details_str}", exc_info=False)
            return None
        except Exception as e:
            logger.error(f"Ein unerwarteter Fehler ist beim Google Drive Upload aufgetreten: {e}", exc_info=True)
            return None
```text
# .env.example
# Kopiere diese Datei zu .env und fülle deine API-Schlüssel und Konfigurationen ein.
# Füge .env zu deiner .gitignore Datei hinzu, um Keys nicht in Git zu committen!

# NewsAPI
NEWSAPI_API_KEY="DEIN_NEWSAPI_SCHLÜSSEL_HIER"

# Google Gemini API (Vertex AI oder AI Studio)
GOOGLE_API_KEY="DEIN_GOOGLE_GEMINI_API_SCHLÜSSEL_HIER"
GEMINI_DEFAULT_MODEL="gemini-1.5-flash-latest" # oder gemini-1.5-pro-latest

# LangSmith (optional, aber empfohlen für Tracing)
LANGCHAIN_API_KEY="DEIN_LANGSMITH_API_SCHLÜSSEL_HIER (ls__...)"
LANGCHAIN_TRACING_V2="true"
LANGCHAIN_ENDPOINT="[https://api.smith.langchain.com](https://api.smith.langchain.com)"
LANGCHAIN_PROJECT="Newsletter-Pipeline-Projekt" # Wähle einen Projektnamen für LangSmith

# Google Cloud Service Account für Google Drive / Calendar etc. (optional, wenn benötigt)
# Der Pfad zur JSON-Schlüsseldatei deines Service Accounts.
# GOOGLE_APPLICATION_CREDENTIALS="/pfad/zu/deiner/service_account_datei.json"

# Google Drive Ordner ID (optional, für Uploads)
# GOOGLE_DRIVE_FOLDER_ID="DEINE_GOOGLE_DRIVE_ORDNER_ID_HIER"

# OpenWeatherMap API Key (optional, für Wetter-Agent)
# OPENWEATHERMAP_API_KEY="DEIN_OPENWEATHERMAP_API_SCHLÜSSEL_HIER"

# Logging Konfiguration (optional)
LOG_LEVEL="INFO" # DEBUG, INFO, WARNING, ERROR, CRITICAL
# LOG_FILE="logs/newsletter_pipeline.log" # Optional: Pfad zur Log-Datei

# Für den AudienceAlignmentAgent
NEWSLETTER_USER_PROFILE="""
Der Leser interessiert sich primär für tiefgehende technische Analysen im Bereich KI,
Softwareentwicklung und Open Source, mit Fokus auf den DACH-Raum und Europa.
Bevorzugt werden Nachrichten über neue LLM-Modelle, ethische Implikationen von KI,
Durchbrüche in der Forschung und relevante wirtschaftliche Entwicklungen im Tech-Sektor.
Weniger relevant sind allgemeine Nachrichten, oberflächliche Produktankündigungen oder
nicht-technologiebezogene Politik und Gesellschaftsthemen, es sei denn, sie haben einen direkten KI-Bezug.
"""
RELEVANCE_THRESHOLD="0.6" # Schwellenwert (0.0-1.0) für Relevanz, um Artikel in den Newsletter aufzunehmen

```text
# requirements.txt
# Liste der Python-Abhängigkeiten für das Projekt.

python-dotenv
requests
# Langchain Kernkomponenten
langchain>=0.1.0 # Stelle sicher, dass eine aktuelle Version verwendet wird
langchain-core>=0.1.0
# Spezifische Langchain Integrationen
langchain-google-genai # Für Gemini
# langchain-openai # Falls du OpenAI als Alternative oder für andere Zwecke nutzen willst
langsmith # Für Tracing und Monitoring mit LangSmith

# Datenvalidierung und -modellierung
pydantic

# Für RSS Feeds (optional, falls benötigt)
feedparser

# Für einfaches HTML-Parsing (optional, falls benötigt)
beautifulsoup4 

# Google Client Libraries (optional, für Google Drive, Calendar etc.)
google-api-python-client
google-auth-httplib2
google-auth-oauthlib

# PDF-Generierung
reportlab # Eine mächtige Bibliothek für PDF-Erstellung
# Alternativen:
# fpdf2
# weasyprint # Gut für HTML zu PDF Konvertierung (benötigt separate Installation von Pango, Cairo etc.)

# Für asynchrone HTTP-Anfragen (optional, für Performance-Optimierung bei vielen API-Calls)
# httpx[http2] 
# asyncio # Teil der Python Standard Library


SyntaxError: invalid syntax (1506332577.py, line 263)